In [6]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc

import keras as k
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D

import cv2
from tqdm import tqdm

from multiprocessing import Pool, cpu_count
from keras.applications import ResNet50
from subprocess import check_output
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
from sklearn.model_selection import train_test_split
from sklearn.metrics import fbeta_score
from keras.callbacks import LambdaCallback, EarlyStopping
import matplotlib.pyplot as plt

In [2]:
df_train = pd.read_csv('inputs/train_v2.csv')
df_train.head(2)

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water


In [8]:
input_dir = 'inputs'
img_size = 256

In [19]:
# model = applications.Xception(weights = "imagenet", include_top=False, input_shape = (img_size, img_size, 3), pooling='avg')
model = applications.ResNet50(include_top=False, weights='imagenet', input_shape = (img_size, img_size, 3))

In [20]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_4 (InputLayer)             (None, 256, 256, 3)   0                                            
____________________________________________________________________________________________________
zero_padding2d_4 (ZeroPadding2D) (None, 262, 262, 3)   0                                            
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 128, 128, 64)  9472                                         
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, 128, 128, 64)  256                                          
___________________________________________________________________________________________

bn3a_branch2a (BatchNormalizatio (None, 32, 32, 128)   512                                          
____________________________________________________________________________________________________
activation_158 (Activation)      (None, 32, 32, 128)   0                                            
____________________________________________________________________________________________________
res3a_branch2b (Conv2D)          (None, 32, 32, 128)   147584                                       
____________________________________________________________________________________________________
bn3a_branch2b (BatchNormalizatio (None, 32, 32, 128)   512                                          
____________________________________________________________________________________________________
activation_159 (Activation)      (None, 32, 32, 128)   0                                            
___________________________________________________________________________________________

____________________________________________________________________________________________________
res4a_branch2a (Conv2D)          (None, 16, 16, 256)   131328                                       
____________________________________________________________________________________________________
bn4a_branch2a (BatchNormalizatio (None, 16, 16, 256)   1024                                         
____________________________________________________________________________________________________
activation_170 (Activation)      (None, 16, 16, 256)   0                                            
____________________________________________________________________________________________________
res4a_branch2b (Conv2D)          (None, 16, 16, 256)   590080                                       
____________________________________________________________________________________________________
bn4a_branch2b (BatchNormalizatio (None, 16, 16, 256)   1024                                

add_59 (Add)                     (None, 16, 16, 1024)  0                                            
____________________________________________________________________________________________________
activation_181 (Activation)      (None, 16, 16, 1024)  0                                            
____________________________________________________________________________________________________
res4e_branch2a (Conv2D)          (None, 16, 16, 256)   262400                                       
____________________________________________________________________________________________________
bn4e_branch2a (BatchNormalizatio (None, 16, 16, 256)   1024                                         
____________________________________________________________________________________________________
activation_182 (Activation)      (None, 16, 16, 256)   0                                            
___________________________________________________________________________________________

____________________________________________________________________________________________________
bn5b_branch2c (BatchNormalizatio (None, 8, 8, 2048)    8192                                         
____________________________________________________________________________________________________
add_63 (Add)                     (None, 8, 8, 2048)    0                                            
____________________________________________________________________________________________________
activation_193 (Activation)      (None, 8, 8, 2048)    0                                            
____________________________________________________________________________________________________
res5c_branch2a (Conv2D)          (None, 8, 8, 512)     1049088                                      
____________________________________________________________________________________________________
bn5c_branch2a (BatchNormalizatio (None, 8, 8, 512)     2048                                

In [11]:
from keras.layers import Add
adds = []
for i, l in enumerate(model.layers):
    if type(l) is Add:
        adds.append(i)
adds

[16, 26, 36, 48, 58, 68, 78, 90, 100, 110, 120, 130, 140, 152, 162, 172]

In [21]:
#Adding classifier
x = model.output
x = Flatten()(x)
x = Dense(256, activation="relu")(x)
x = BatchNormalization()(x)
x = Dropout(0.25)(x)
x = Dense(17, activation="sigmoid")(x)

# creating the final model 
model_final = Model(inputs = model.input, outputs = x)
model_final.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_4 (InputLayer)             (None, 256, 256, 3)   0                                            
____________________________________________________________________________________________________
zero_padding2d_4 (ZeroPadding2D) (None, 262, 262, 3)   0                                            
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 128, 128, 64)  9472                                         
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, 128, 128, 64)  256                                          
___________________________________________________________________________________________

bn3a_branch2a (BatchNormalizatio (None, 32, 32, 128)   512                                          
____________________________________________________________________________________________________
activation_158 (Activation)      (None, 32, 32, 128)   0                                            
____________________________________________________________________________________________________
res3a_branch2b (Conv2D)          (None, 32, 32, 128)   147584                                       
____________________________________________________________________________________________________
bn3a_branch2b (BatchNormalizatio (None, 32, 32, 128)   512                                          
____________________________________________________________________________________________________
activation_159 (Activation)      (None, 32, 32, 128)   0                                            
___________________________________________________________________________________________

____________________________________________________________________________________________________
res4a_branch2a (Conv2D)          (None, 16, 16, 256)   131328                                       
____________________________________________________________________________________________________
bn4a_branch2a (BatchNormalizatio (None, 16, 16, 256)   1024                                         
____________________________________________________________________________________________________
activation_170 (Activation)      (None, 16, 16, 256)   0                                            
____________________________________________________________________________________________________
res4a_branch2b (Conv2D)          (None, 16, 16, 256)   590080                                       
____________________________________________________________________________________________________
bn4a_branch2b (BatchNormalizatio (None, 16, 16, 256)   1024                                

add_59 (Add)                     (None, 16, 16, 1024)  0                                            
____________________________________________________________________________________________________
activation_181 (Activation)      (None, 16, 16, 1024)  0                                            
____________________________________________________________________________________________________
res4e_branch2a (Conv2D)          (None, 16, 16, 256)   262400                                       
____________________________________________________________________________________________________
bn4e_branch2a (BatchNormalizatio (None, 16, 16, 256)   1024                                         
____________________________________________________________________________________________________
activation_182 (Activation)      (None, 16, 16, 256)   0                                            
___________________________________________________________________________________________

____________________________________________________________________________________________________
bn5b_branch2c (BatchNormalizatio (None, 8, 8, 2048)    8192                                         
____________________________________________________________________________________________________
add_63 (Add)                     (None, 8, 8, 2048)    0                                            
____________________________________________________________________________________________________
activation_193 (Activation)      (None, 8, 8, 2048)    0                                            
____________________________________________________________________________________________________
res5c_branch2a (Conv2D)          (None, 8, 8, 512)     1049088                                      
____________________________________________________________________________________________________
bn5c_branch2a (BatchNormalizatio (None, 8, 8, 512)     2048                                